<a href="https://colab.research.google.com/github/wonjae124/2021_Lab_project/blob/main/AddNoise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import skimage.util.noise as noise #Denoise 패키지인 Sckit.image의 util모듈. return 값은 numpy로 다루는 ndarray임. https://scikit-image.org/docs/stable/api/skimage.util.html#random-noise 
from torchvision import datasets, transforms

In [13]:
train_set = datasets.MNIST('./data',train=True, download=True,transform=transforms.ToTensor)
test_set = datasets.MNIST('./data',train=False, download=False,transform=transforms.ToTensor)

In [14]:
train_set_arr=train_set.data.numpy() #numpy 자료형(ndarray) 변환.
test_set_arr=test_set.data.numpy()

In [15]:
snp_train_set_arr1 = noise.random_noise(train_set_arr,mode='s&p',amount=0.2) #함수 random_noise를 사용한다. speckle도 추가 가능
snp_train_set_arr2 = noise.random_noise(train_set_arr,mode='s&p',amount=0.5)
snp_train_set_arr3 = noise.random_noise(train_set_arr,mode='s&p',amount=0.7)

gn_train_set_arr1 = noise.random_noise(train_set_arr,mode='gaussian',mean=0.2) #gaussian,speckle noise의 비율을 결정하는 파라미터인 mean이다. default는 0이다.
gn_train_set_arr2 = noise.random_noise(train_set_arr,mode='gaussian',mean=0.5)
gn_train_set_arr3 = noise.random_noise(train_set_arr,mode='gaussian',mean=0.7)

snp_test_set_arr = noise.random_noise(test_set_arr,mode='s&p') #amount는 noise의 propotional을 뜻함. [0,1]로 0~1이며 default=0.5
gn_test_set_arr = noise.random_noise(test_set_arr,mode='gaussian')
gn2 = noise.random_noise(test_set_array,mode='s&p',amount=0.2) 

In [ ]:
#아직 못 끝내고 필요한 것 -> 1. 노이즈가 들어갔는지 이미지 띄워서 확인하기 
#궁금한 것 -> ndarray형식을 이미지로 바꾼 후에 바로 SSIM으로 표현 할 수 있는지 궁금함
